In [ ]:
# ============================================================
# Credit Card Fraud Detection - Ensemble + Full SHAP Explainability (incl. interactions)
# ============================================================
import os, json, joblib, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    roc_curve, precision_recall_curve, average_precision_score,
)

import lightgbm as lgb
import shap

warnings.filterwarnings("ignore")
plt.rcParams["figure.dpi"] = 150

# ============================================================
# Paths
# ============================================================
BASE = Path("/kaggle/working")
DATA = Path("/kaggle/input/creditcardfraud/creditcard.csv")
OUT = BASE / "out/fraud_model"
PROC = BASE / "data/fraud/processed"
OUT.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)

# ============================================================
# Load Data
# ============================================================
df = pd.read_csv(DATA)
print("Data shape:", df.shape)
print("Columns:", df.columns.tolist())

# Features and target
X = df.drop("Class", axis=1).copy()
y = df["Class"].copy()

# Train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Scale Amount; drop Time
scaler_amount = StandardScaler()
X_train = X_train.copy()
X_test = X_test.copy()
X_train.loc[:, "normAmount"] = scaler_amount.fit_transform(X_train[["Amount"]])
X_test.loc[:,  "normAmount"] = scaler_amount.transform(X_test[["Amount"]])

X_train.drop(["Time", "Amount"], axis=1, inplace=True)
X_test.drop(["Time", "Amount"], axis=1, inplace=True)

# Save feature order & scaler
feature_order = list(X_train.columns)
json.dump(feature_order, open(PROC / "feature_order.json", "w"))
joblib.dump(scaler_amount, PROC / "scaler_normAmount.pkl")
print("Processed artifacts saved to:", PROC)
print(os.listdir(PROC))

# ============================================================
# Train Ensemble Models
# ============================================================
def train_seed(seed):
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_test, label=y_test)

    pos_count = int(y_train.sum())
    neg_count = len(y_train) - pos_count
    spw = neg_count / max(1, pos_count)

    params = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": 0.01,
        "num_leaves": 31,
        "min_data_in_leaf": 50,
        "feature_fraction": 0.8,
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "verbosity": -1,
        "seed": seed,
        "scale_pos_weight": float(min(50.0, spw)),  # clip for stability
    }
    model = lgb.train(
        params,
        dtrain,
        valid_sets=[dval],
        num_boost_round=2000,
        callbacks=[lgb.early_stopping(100), lgb.log_evaluation(100)],
    )
    return model

seeds = [11, 22, 33]
models = [train_seed(s) for s in seeds]

# Save models
for i, m in enumerate(models):
    joblib.dump(m, OUT / f"lgb_fraud_ens_seed_{seeds[i]}.pkl")

# ============================================================
# Ensemble Predictions & Metrics
# ============================================================
probs = np.mean([m.predict(X_test, num_iteration=m.best_iteration) for m in models], axis=0)
y_pred_05 = (probs >= 0.5).astype(int)

metrics = {
    "accuracy@0.5": float(accuracy_score(y_test, y_pred_05)),
    "precision@0.5": float(precision_score(y_test, y_pred_05, zero_division=0)),
    "recall@0.5": float(recall_score(y_test, y_pred_05, zero_division=0)),
    "f1@0.5": float(f1_score(y_test, y_pred_05, zero_division=0)),
    "roc_auc": float(roc_auc_score(y_test, probs)),
    "avg_precision": float(average_precision_score(y_test, probs)),
}
print("Metrics:", json.dumps(metrics, indent=2))

# Best F1 threshold search
ths = np.linspace(0, 1, 1001)
f1s = [f1_score(y_test, (probs >= t).astype(int), zero_division=0) for t in ths]
best_thr = float(ths[int(np.argmax(f1s))])
y_pred_best = (probs >= best_thr).astype(int)

metrics.update({
    "best_threshold": best_thr,
    "accuracy@best": float(accuracy_score(y_test, y_pred_best)),
    "precision@best": float(precision_score(y_test, y_pred_best, zero_division=0)),
    "recall@best": float(recall_score(y_test, y_pred_best, zero_division=0)),
    "f1@best": float(f1_score(y_test, y_pred_best, zero_division=0)),
})

with open(OUT / "metrics_ensemble.json", "w") as f:
    json.dump(metrics, f, indent=2)

# ============================================================
# Plots: ROC, PR, Confusion Matrices
# ============================================================
# ROC
fpr, tpr, _ = roc_curve(y_test, probs)
plt.figure()
plt.plot(fpr, tpr, label=f"Ensemble AUC={metrics['roc_auc']:.3f}")
plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend()
plt.title("ROC Curve (Ensemble)")
plt.savefig(OUT / "ensemble_roc.png")
plt.close()

# PR
prec, rec, _ = precision_recall_curve(y_test, probs)
plt.figure()
plt.plot(rec, prec, label=f"AP={metrics['avg_precision']:.3f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend()
plt.title("Precision-Recall Curve (Ensemble)")
plt.savefig(OUT / "ensemble_pr.png")
plt.close()

# Confusion matrices
def plot_cm(y_true, y_pred, title, fname):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure()
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title(title)
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(OUT / fname)
    plt.close()

plot_cm(y_test, y_pred_05, "Confusion Matrix (thr=0.5)", "cm_thr_0.5.png")
plot_cm(y_test, y_pred_best, f"Confusion Matrix (thr={best_thr:.3f})", "cm_thr_best.png")
json.dump({"best_threshold": best_thr}, open(OUT / "best_threshold.json", "w"))

print("Artifacts saved to:", OUT)
print(sorted(os.listdir(OUT)))

# ============================================================
# SHAP Explainability (values, comparisons)
# ============================================================
def _shap_values_binary(explainer, X_):
    """Return SHAP values for positive class consistently across SHAP versions."""
    sv = explainer.shap_values(X_)
    if isinstance(sv, list):  # [class0, class1]
        return sv[1]
    return sv

def _base_value_binary(explainer):
    """Return scalar base value for positive class robustly."""
    bv = explainer.expected_value
    # SHAP may return list/array for [neg, pos]; pick pos if length 2
    if isinstance(bv, (list, np.ndarray)) and len(np.atleast_1d(bv)) >= 2:
        return float(np.atleast_1d(bv)[1])
    # else assume scalar
    return float(bv)

# Use first model for classical SHAP plots
explainer0 = shap.TreeExplainer(models[0])
sv0 = _shap_values_binary(explainer0, X_test)
base_val = _base_value_binary(explainer0)

# SHAP summary (beeswarm)
plt.figure()
shap.summary_plot(sv0, X_test, plot_type="dot", show=False)
plt.savefig(OUT / "shap_summary_beeswarm.png", bbox_inches="tight")
plt.close()

# SHAP bar importance (top 20)
plt.figure()
shap.summary_plot(sv0, X_test, plot_type="bar", max_display=20, show=False)
plt.savefig(OUT / "shap_summary_bar.png", bbox_inches="tight")
plt.close()

# Single-sample force plot (robust to SHAP>=0.40)
shap.initjs()
sample_idx = int(np.where(y_test.values == 1)[0][0]) if y_test.sum() > 0 else 0
# Static PNG via matplotlib mode
plt.figure()
shap.force_plot(base_val, sv0[sample_idx, :], X_test.iloc[sample_idx, :], matplotlib=True, show=False)
plt.savefig(OUT / "shap_force_sample.png", bbox_inches="tight")
plt.close()

# Compare importances across ensemble
def mean_abs_shap(sv):
    return np.abs(sv).mean(axis=0)

sv_list = []
for m in models:
    e = shap.TreeExplainer(m)
    sv = _shap_values_binary(e, X_test)
    sv_list.append(sv)

sv_mean = np.mean(sv_list, axis=0)  # (n_samples, n_features)

df_imp = pd.DataFrame(
    {f"Model{i+1}": mean_abs_shap(sv_list[i]) for i in range(len(sv_list))},
    index=X_test.columns,
)
df_imp["Ensemble"] = mean_abs_shap(sv_mean)

df_top = df_imp / df_imp.max()
top20 = df_top.sort_values("Ensemble", ascending=False).head(20)
plt.figure(figsize=(12, 6))
top20.plot(kind="bar")
plt.title("Top-20 Feature Importance Comparison (SHAP mean |value|)")
plt.tight_layout()
plt.savefig(OUT / "shap_comparison_top20.png")
plt.close()

# Correlation of importances across models
plt.figure(figsize=(6, 5))
sns.heatmap(df_imp.corr(), annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation of SHAP Importances Across Models")
plt.tight_layout()
plt.savefig(OUT / "shap_importance_correlation.png")
plt.close()

# ============================================================
# SHAP INTERACTION VALUES (subsample for speed)
# ============================================================
# NOTE: SHAP interaction values can be heavy. We'll subsample to up to 2000 rows.
max_inter_rows = 2000
X_inter = X_test.sample(n=max_inter_rows, random_state=42) if len(X_test) > max_inter_rows else X_test.copy()

explainer_int = shap.TreeExplainer(models[0])
sv_int = explainer_int.shap_interaction_values(X_inter)
if isinstance(sv_int, list):
    sv_int = sv_int[1]  # (n, f, f)

# Mean absolute interaction strength matrix
mean_abs_int = np.abs(sv_int).mean(axis=0)  # (f, f)
feat_names = X_inter.columns

# Heatmap of interaction strengths
plt.figure(figsize=(8, 7))
sns.heatmap(mean_abs_int, xticklabels=feat_names, yticklabels=feat_names, cmap="viridis")
plt.title("SHAP Interaction Values (mean |interaction|)")
plt.tight_layout()
plt.savefig(OUT / "shap_interaction_heatmap.png")
plt.close()

# Extract top pairwise interactions (upper triangle)
tri_idx = np.triu_indices_from(mean_abs_int, k=1)
pairs = [((feat_names[i], feat_names[j]), mean_abs_int[i, j]) for i, j in zip(*tri_idx)]
pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
top_pairs = pairs_sorted[:3]  # top-3 interactions

# Dependence plots for top-3 interactions
for k, ((f_main, f_int), strength) in enumerate(top_pairs, start=1):
    plt.figure()
    shap.dependence_plot(
        f_main,
        sv0 if sv0.shape[0] == X_test.shape[0] else _shap_values_binary(explainer0, X_test),
        X_test,
        interaction_index=f_int,
        show=False,
    )
    plt.title(f"SHAP Dependence: {f_main} (interaction w/ {f_int})")
    plt.tight_layout()
    plt.savefig(OUT / f"shap_dependence_{k}_{f_main}_x_{f_int}.png")
    plt.close()

# Also save a small JSON summary of top interactions
top_inter_json = [{"main": a, "interaction": b, "mean_abs_strength": float(s)} for (a, b), s in top_pairs]
json.dump(top_inter_json, open(OUT / "shap_top_interactions.json", "w"), indent=2)

# ============================================================
# Final prints
# ============================================================
print("\n=== FINAL METRICS ===")
print(json.dumps(metrics, indent=2))
print("\nArtifacts saved to:", OUT)
print(sorted(os.listdir(OUT)))


In [ ]:
# ============================================================
# Save X_test and y_test for Gradio Dashboard
# ============================================================
X_test.to_parquet(PROC/"X_test.parquet", index=False)
joblib.dump(y_test, PROC/"y_test.pkl")

print("Saved X_test and y_test to:", PROC)
print(os.listdir(PROC))


In [ ]:
# ============================================================
# Fraud Detection Gradio Dashboard (fixed image return: numpy)
# ============================================================
import gradio as gr
import shap, json, joblib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from io import BytesIO
from pathlib import Path
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from PIL import Image

# === Load Artifacts ===
OUT = Path("/kaggle/working/out/fraud_model")
PROC = Path("/kaggle/working/data/fraud/processed")

feat_order_loaded = json.load(open(PROC/"feature_order.json"))
scaler_loaded = joblib.load(PROC/"scaler_normAmount.pkl")
models = [joblib.load(OUT/f) for f in ["lgb_fraud_ens_seed_11.pkl",
                                       "lgb_fraud_ens_seed_22.pkl",
                                       "lgb_fraud_ens_seed_33.pkl"]]
best_thr = json.load(open(OUT/"best_threshold.json"))["best_threshold"]

X_test = pd.read_parquet(PROC/"X_test.parquet")
y_test = joblib.load(PROC/"y_test.pkl")

# === Helpers ===
def fig_to_numpy():
    """Convert current matplotlib figure to numpy array (RGB)."""
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    img = Image.open(buf).convert("RGB")
    arr = np.array(img)
    plt.close()
    return arr

# === Prediction Helper ===
def predict_sample(sample_dict, scaler, feature_order, models, thr):
    df_in = pd.DataFrame([sample_dict])
    if "Time" in df_in: df_in = df_in.drop("Time", axis=1)
    if "Amount" in df_in:
        df_in["normAmount"] = scaler.transform(df_in[["Amount"]])
        df_in = df_in.drop("Amount", axis=1)
    for col in feature_order:
        if col not in df_in: df_in[col] = 0
    df_in = df_in[feature_order]
    # Ensemble prob
    probs = np.mean([m.predict(df_in, num_iteration=m.best_iteration) for m in models], axis=0)
    prob = float(probs[0])
    label = int(prob >= thr)
    return df_in, {"probability": prob, "label": label}

# === SHAP explainer ===
explainer = shap.TreeExplainer(models[0])

def shap_plot(df_in):
    shap_vals = explainer.shap_values(df_in)
    if isinstance(shap_vals, list):  # binary -> take class 1
        shap_vals = shap_vals[1]
    plt.figure(figsize=(8,4))
    shap.summary_plot(shap_vals, df_in, plot_type="bar", show=False)
    return fig_to_numpy()

def shap_summary(X):
    shap_vals = explainer.shap_values(X[:200])
    if isinstance(shap_vals, list):
        shap_vals = shap_vals[1]
    plt.figure(figsize=(8,5))
    shap.summary_plot(shap_vals, X[:200], show=False)
    return fig_to_numpy()

# === Gradio predict functions ===
example_input = json.dumps(X_test.iloc[0].to_dict(), indent=2)

def gradio_predict(json_str):
    try:
        sample = json.loads(json_str)
    except:
        return {"error": "Invalid JSON input"}, None
    df_in, out = predict_sample(sample, scaler_loaded, feat_order_loaded, models, best_thr)
    shap_img = shap_plot(df_in)
    return out, shap_img

def gradio_batch(file_obj):
    df = pd.read_csv(file_obj.name)
    preds = []
    for i, row in df.iterrows():
        df_in, out = predict_sample(row.to_dict(), scaler_loaded, feat_order_loaded, models, best_thr)
        preds.append(out)
    pred_df = pd.DataFrame(preds)
    tmp_path = "/kaggle/working/batch_predictions.csv"
    pred_df.to_csv(tmp_path, index=False)
    global_shap_img = shap_summary(X_test)
    return pred_df, tmp_path, global_shap_img

# === Metrics Dashboard ===
def get_metrics_and_cm():
    probs = np.mean([m.predict(X_test, num_iteration=m.best_iteration) for m in models], axis=0)
    y_pred = (probs >= best_thr).astype(int)
    report = classification_report(y_test, y_pred, output_dict=True)
    auc = roc_auc_score(y_test, probs)

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure()
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted"); plt.ylabel("True")
    cm_img = fig_to_numpy()

    return {
        "Accuracy": report["accuracy"],
        "Precision": report["1"]["precision"],
        "Recall": report["1"]["recall"],
        "F1": report["1"]["f1-score"],
        "AUC": auc
    }, cm_img

# === Gradio UI ===
with gr.Blocks() as demo:
    gr.Markdown("## Fraud Detection Dashboard 🚀 (LightGBM Ensemble + SHAP + Metrics)")
    
    with gr.Tab("🔍 Single Transaction"):
        inp = gr.Textbox(label="Transaction JSON", lines=12, value=example_input)
        out_json = gr.JSON(label="Prediction")
        out_img = gr.Image(type="numpy", label="SHAP Feature Importance")
        btn = gr.Button("Predict")
        btn.click(fn=gradio_predict, inputs=inp, outputs=[out_json, out_img])
    
    with gr.Tab("📂 Batch Prediction"):
        file_in = gr.File(label="Upload CSV", file_types=[".csv"])
        out_tbl = gr.Dataframe(label="Batch Predictions")
        out_file = gr.File(label="Download Predictions (CSV)")
        out_global = gr.Image(type="numpy", label="Global SHAP Summary (Beeswarm)")
        file_in.upload(fn=gradio_batch, inputs=file_in, outputs=[out_tbl, out_file, out_global])

    with gr.Tab("📊 Metrics Dashboard"):
        btn2 = gr.Button("Compute Test Metrics + Confusion Matrix")
        out_metrics = gr.JSON(label="Metrics")
        out_cm = gr.Image(type="numpy", label="Confusion Matrix")
        btn2.click(fn=get_metrics_and_cm, inputs=None, outputs=[out_metrics, out_cm])

# Launch
demo.launch(share=True, debug=True)


In [ ]:
# ============================================================
# Save Final Fraud Detection Model (Ensemble + Metadata) - No Calibration
# ============================================================
import joblib, json
from pathlib import Path

OUT = Path("/kaggle/working/out/fraud_model")
PROC = Path("/kaggle/working/data/fraud/processed")

final_model_bundle = {
    "models": [
        joblib.load(OUT/"lgb_fraud_ens_seed_11.pkl"),
        joblib.load(OUT/"lgb_fraud_ens_seed_22.pkl"),
        joblib.load(OUT/"lgb_fraud_ens_seed_33.pkl")
    ],
    "calibrator": None,  # no calibration
    "scaler": joblib.load(PROC/"scaler_normAmount.pkl"),
    "feature_order": json.load(open(PROC/"feature_order.json")),
    "best_threshold": json.load(open(OUT/"best_threshold.json"))["best_threshold"],
    "metrics": json.load(open(OUT/"metrics_ensemble.json"))
}

joblib.dump(final_model_bundle, OUT/"final_fraud_model.pkl")
print("✅ Final model bundle saved (no calibration):", OUT/"final_fraud_model.pkl")


In [ ]:
from sklearn.linear_model import LogisticRegression

# Get ensemble raw probs
probs = np.mean([m.predict(X_test, num_iteration=m.best_iteration) for m in models], axis=0).reshape(-1,1)

# Train calibration model
calib = LogisticRegression(max_iter=1000)
calib.fit(probs, y_test)

# Save calibration model
joblib.dump(calib, OUT/"calibration_lr.pkl")
print("✅ Calibration model saved")

# Then re-run the final bundle saving code


In [ ]:
bundle = joblib.load("/kaggle/working/out/fraud_model/final_fraud_model.pkl")

models = bundle["models"]
calib = bundle["calibrator"]
scaler = bundle["scaler"]
feat_order = bundle["feature_order"]
thr = bundle["best_threshold"]

print("Loaded final model with best threshold:", thr)
